## 이미지 크롤링(파이리, 이브이)

In [ ]:
#! pip install tqdm

In [4]:
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import os, sys, warnings # 실행 중인지 아닌지를 확인하는 모듈

In [5]:
from selenium.webdriver.common.by import By  # 제어를 위한 셀렉트 용 모듈
from urllib import request   # 이미지 다운로드에 사용

In [26]:
browser = Chrome('c:/pydata/chromedriver.exe')
time.sleep(1)

#### 키워드 입력 및 URL 생성

keyword = input('검색어 입력 : ')

url = 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query=' + keyword
browser.get(url)

#### 페이지 이동
# 사용자가 요구한 스크롤 다운 횟수 확인 필요

count_down = int(input('스크롤 다운 횟수(6) : '))

if count_down > 6 :
    count_down = 6

# windows.scrollTo(0, Height)  :  스크롤을 아래쪽으로 이동
# Height 대신 document.body.scrollHeight를 입력하면 페이지 끝으로 이동하는 것!
for i in range(count_down):
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1)

C:\Users\sujin\AppData\Local\Temp\ipykernel_22636\1256623951.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome('c:/pydata/chromedriver.exe')


검색어 입력 : 이브이
스크롤 다운 횟수(6) : 6


### 화면에서 이미지 추출하기
1. 드라이브의 소스코드 받아오기
2. 이미지가 있는 태그 찾기
3. 태그에서 이미지 소스['src'] 받아오기
4. src 주소를 이용해 이미지 다운로드 하기

In [27]:
# 드라이브로 소스코드 가져오기
html = browser.page_source
soup = bs(html, 'html.parser')

In [28]:
# 이미지 대표 태그 가져오기 (find_all -> 리스트)
tag_img = soup.find_all('div', class_='thumb')
tag_img[4].find('img')['src']

'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7'

In [29]:
tag_img[4].find('img')

<img alt="[포켓몬 정보] 이브이의 진화체" class="_image _listImage" data-lazy-src="https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMDExMTVfMTg2%2FMDAxNjA1NDQyNDE3NTk3.ruJs0gCZaoLrKaZLNZQF2mAIM8ndCzxp5gtW3U8NyiYg.z1my6OjzigVFZcRRV8RkQcCNGyiwgdAcmHrBde6tW2gg.JPEG.abcog02%2FCollageMaker%25A3%25DF20200725%25A3%25DF225806052.jpg&amp;type=l340_165" data-org-height="165" data-org-width="340" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="height: 106px;"/>

In [30]:
len(tag_img)

531

In [31]:
tag_src = []
for tag in tag_img:
    if tag.find('img')['src'].startswith('https://'):
        tag_src.append(tag.find('img')['src'])
    else:
        tag_src.append(tag.find('img')['data-lazy-src'])
    
print(len(tag_src))

browser.close()

531


In [32]:
print(tag_src[4])

https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMDExMTVfMTg2%2FMDAxNjA1NDQyNDE3NTk3.ruJs0gCZaoLrKaZLNZQF2mAIM8ndCzxp5gtW3U8NyiYg.z1my6OjzigVFZcRRV8RkQcCNGyiwgdAcmHrBde6tW2gg.JPEG.abcog02%2FCollageMaker%25A3%25DF20200725%25A3%25DF225806052.jpg&type=l340_165


### 검색어를 이용해 폴더 생성 후 전체 이미지 저장
- os.makedirs('디렉토리경로/생성 폴더명') -> 폴더 생성
- 기존 폴더 존재시 기존 폴더 이름 뒤에 1, 2, ...와 같이 번호 붙여 생성
- 해당 폴더에 저장, 없으면 생성 후 저장

### 이미지를 저장 할 대표 폴더 생성[images]
- imges 폴더에 검색 키워드를 이용해 폴더 생성 후 저장
- 키워드와 동일한 폴더가 있는 경우 1씩 증가시키면서 저장

- 이미지 저장 (저장 이미지 이름 뒤에 번호 붙이기)

In [33]:
i = 0
fdir = 'c:/pydata/poketmon/이브이/'

for link in tqdm(tag_src):          
    request.urlretrieve(link, fdir +'이브이' +f'{i}.jpg')
    i += 1

100%|████████████████████████████████████████████████████████████████████████████████| 531/531 [00:34<00:00, 15.58it/s]


# 구글 이미지 크롤링
1. 스크롤 바를 이용해 검색한 기본 이미지 검색 키워드 폴더 생성 후 저장
2. [추가 미션] 스크롤바 이동 후 더보기 버튼이 나오면 클릭 후 나타난 모든 이미지 저장 

In [34]:
keyword = input('키워드를 입력하세요: ')
url = 'https://www.google.com/search?q='+ keyword+'&aqs=chrome.0.69i59j46i20i263i433i512j0i131i395i433i512l3j46i131i199i395i424i433i465i466i512j46i131i395i433i512j46i131i395i408i410i424i433i512j0i395i433i512j0i395i512.1002j1j15&sourceid=chrome&ie=UTF-8'

driver = Chrome('c:/pydata/chromedriver.exe')
driver.get(url)

# 이미지 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="hdtb-msb"]/div[1]/div/div[2]/a').click()
time.sleep(1)
# 스크롤 내리기 (네 번?)

count_down = int(input('스크롤 다운 횟수 입력(최대 4회) : '))

if count_down > 4 :
    count_down = 4

# windows.scrollTo(0, Height)  :  스크롤을 아래쪽으로 이동
# Height 대신 document.body.scrollHeight를 입력하면 페이지 끝으로 이동하는 것!
for i in range(count_down):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1)

키워드를 입력하세요: 이브이


C:\Users\sujin\AppData\Local\Temp\ipykernel_22636\1720839674.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome('c:/pydata/chromedriver.exe')


스크롤 다운 횟수 입력(최대 4회) : 4


- 결과 더보기 버튼 클릭

In [36]:
# 결 더보기 버튼 클릭

driver.find_element(By.XPATH, '//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input').click()
time.sleep(1)

count_down = int(input('스크롤 다운 횟수 입력(파이리2,이브이1) : '))

if count_down > 4 :
    count_down = 4

for i in range(count_down):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1)

스크롤 다운 횟수 입력(파이리2,이브이1) : 1


In [38]:
html = driver.page_source
soup = bs(html, 'html.parser')
img_lst = soup.find_all('div', class_='isv-r PNCib MSM1fd BUooTd')

In [39]:
len(img_lst)

594

In [40]:
i = 531

for link in tqdm(img_lst):          
    try :
        request.urlretrieve(link.find('img')['src'], fdir + keyword +f'{i}.jpg')
    except:
        request.urlretrieve(link.find('img')['data-src'], fdir + keyword +f'{i}.jpg')
    i += 1

100%|████████████████████████████████████████████████████████████████████████████████| 594/594 [02:10<00:00,  4.54it/s]
